In [ ]:
import numpy as np
import pandas as pd

file_path= 'Behavioral Data/Survey_Visual_2019-07-23-15-41-47.csv'
data = pd.read_csv(file_path, skiprows=1)  # <-- Skip the first metadata row
data

In [ ]:
# Convert time to seconds
def time_to_seconds(time_str):
    """
    Convert time string like '40:10.7' or '01:02:03.456' into seconds.
    """
    parts = time_str.strip().split(':')
    if len(parts) == 3:
        h, m, s = map(float, parts)
        return h * 3600 + m * 60 + s
    elif len(parts) == 2:
        m, s = map(float, parts)
        return m * 60 + s
    else:
        return np.nan
data['Time_sec'] = data['Time'].astype(str).apply(time_to_seconds)

# Calculate the duration of session
total_duration = data['Time_sec'].max() - data['Time_sec'].min()

# Get total number of trials
trial_numbers = data['Trial'].unique()
total_trials = len(trial_numbers)

# Filter successful trials (Event == "Target Reached")
successful_trials = data[data['Event'] == 'Target Reached']
successful_trial_numbers = successful_trials['Trial'].unique()
successful_trials_count = len(successful_trial_numbers)

# Compute reward frequency (successful trials per minute)
reward_frequency = successful_trials_count / (total_duration / 60)

# Calculate duration of each trial
trial_durations = []
for trial in trial_numbers:
    trial_times = data[data['Trial'] == trial]['Time_sec']
    duration = trial_times.max() - trial_times.min()
    trial_durations.append(duration)

average_trial_time = np.mean(trial_durations)
std_trial_time = np.std(trial_durations)

# Print results
print(f"Total experiment duration (seconds): {total_duration:.2f}")
print(f"Total number of trials: {total_trials}")
print(f"Number of successful trials: {successful_trials_count}")
print(f"Reward frequency (rewards per minute): {reward_frequency:.2f}")
print(f"Average trial duration (seconds): {average_trial_time:.2f}")
print(f"Standard deviation of trial durations (seconds): {std_trial_time:.2f}")

In [ ]:
import pandas as pd
import numpy as np
import os

# List of behavioral CSV files
file_names = [
    'Behavioral data/Survey_Visual_2019-07-23-15-41-47.csv','Behavioral data/Survey_Visual_2019-07-23-16-11-36.csv','Behavioral data/Survey_Visual_2019-07-23-18-22-01.csv','Behavioral data/Survey_Visual_2019-07-23-18-43-55.csv'
    # Add other file names here
]

# Result table will be stored in a list of dictionaries
results = []


In [ ]:

for file in file_names:
    try:
        data = pd.read_csv(file, skiprows=1)
        data.columns = data.columns.str.strip().str.lower()

        if not all(col in data.columns for col in ['time', 'trial', 'event']):
            print(f"Skipping {file}: Missing required columns.")
            continue

        data['time_sec'] = data['time'].astype(str).apply(time_to_seconds)

        # Calculate stats
        total_duration = data['time_sec'].max() - data['time_sec'].min()
        trial_numbers = data['trial'].unique()
        total_trials = len(trial_numbers)

        successful_trials = data[data['event'] == 'Target Reached']
        successful_trial_numbers = successful_trials['trial'].unique()
        successful_trials_count = len(successful_trial_numbers)

        successful_rate = successful_trials_count / total_trials if total_trials > 0 else np.nan
        reward_frequency = successful_trials_count / (total_duration / 60) if total_duration > 0 else np.nan

        # Trial durations
        trial_durations = []
        for trial in trial_numbers:
            trial_times = data[data['trial'] == trial]['time_sec']
            duration = trial_times.max() - trial_times.min()
            trial_durations.append(duration)

        average_trial_time = np.mean(trial_durations)
        std_trial_time = np.std(trial_durations)

        results.append({
            'FileName': os.path.basename(file),
            'TotalDuration': total_duration,
            'TotalTrials': total_trials,
            'SuccessfulTrialNumber': successful_trials_count,
            'SuccessfulRate': successful_rate,
            'RewardFrequencyPerMin': reward_frequency,
            'AverageTrialTime': average_trial_time,
            'StdTrialTime': std_trial_time
        })

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results)
results_df.to_csv('reward_analysis_results_extended.csv', index=False)

print("Saved results to reward_analysis_results_extended.csv")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the behavioral data
file_path = 'Behavioral data/Survey_Visual_2019-07-23-15-41-47.csv'
data = pd.read_csv(file_path, skiprows=1)
data.columns = data.columns.str.strip().str.lower()

# Convert trial to int and event to lowercase (standardize)
data['trial'] = data['trial'].astype(int)
data['event'] = data['event'].str.strip().str.lower()

# Ensure required columns exist
required_cols = ['trial', 'event', 'position']
if not all(col in data.columns for col in required_cols):
    raise ValueError("Missing one or more required columns: trial, event, position")

# Start plotting
plt.figure(figsize=(10, 8))
plt.title("1D Trial Trajectories (Green: Success, Red: Fail)")
plt.xlabel("Position (Right = +, Left = -)")
plt.ylabel("Trial Number")

trial_numbers = data['trial'].unique()
for i, trial_id in enumerate(trial_numbers):
    trial_data = data[data['trial'] == trial_id]

    events = trial_data['event'].values

    if 'target reached' in events:
        start_idx = trial_data[trial_data['event'] == 'control applied'].index.min()
        end_idx = trial_data[trial_data['event'] == 'target reached'].index.min()
        if pd.notna(start_idx) and pd.notna(end_idx) and end_idx > start_idx:
            traj = trial_data.loc[start_idx:end_idx, 'position']
            plt.plot(traj.values, [i]*len(traj), 'g-', linewidth=1.5)

    elif 'timeout start' in events:
        start_idx = trial_data[trial_data['event'] == 'control applied'].index.min()
        end_idx = trial_data[trial_data['event'] == 'timeout start'].index.min()
        if pd.notna(start_idx) and pd.notna(end_idx) and end_idx > start_idx:
            traj = trial_data.loc[start_idx:end_idx, 'position']
            plt.plot(traj.values, [i]*len(traj), 'r-', linewidth=1.5)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data
file_path = 'Behavioral data/Survey_Visual_2019-07-23-15-41-47.csv'
data = pd.read_csv(file_path, skiprows=1)
data.columns = data.columns.str.strip().str.lower()

# Standardize formatting
data['event'] = data['event'].str.strip().str.lower()
data['trial'] = data['trial'].astype(int)

# Time converter
def time_to_seconds(time_str):
    parts = time_str.strip().split(':')
    if len(parts) == 3:
        h, m, s = map(float, parts)
        return h * 3600 + m * 60 + s
    elif len(parts) == 2:
        m, s = map(float, parts)
        return m * 60 + s
    return np.nan

data['time_sec'] = data['time'].astype(str).apply(time_to_seconds)

# Plot setup
plt.figure(figsize=(12, 8))
plt.title("2D Trajectories (Time vs Position per Trial)")
plt.xlabel("Position")
plt.ylabel("Time (sec, relative to Control Applied)")

trial_numbers = data['trial'].unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(trial_numbers)))

# For legend
legend_labels = []

for i, trial_id in enumerate(trial_numbers):
    trial_data = data[data['trial'] == trial_id]
    events = trial_data['event'].values

    if 'target reached' in events:
        start_idx = trial_data[trial_data['event'] == 'control applied'].index.min()
        end_idx = trial_data[trial_data['event'] == 'target reached'].index.min()
        linestyle = '-'
    elif 'timeout start' in events:
        start_idx = trial_data[trial_data['event'] == 'control applied'].index.min()
        end_idx = trial_data[trial_data['event'] == 'timeout start'].index.min()
        linestyle = '--'
    else:
        continue

    if pd.notna(start_idx) and pd.notna(end_idx) and end_idx > start_idx:
        segment = trial_data.loc[start_idx:end_idx]
        relative_time = segment['time_sec'] - segment['time_sec'].iloc[0]
        position = segment['position'].values

        # Plot and add label
        label = f"Trial {trial_id}"
        plt.plot(position, relative_time, linestyle, color=colors[i], linewidth=1.5, label=label)
        legend_labels.append(label)

# Add legend outside the plot
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title='Trials')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Load behavioral data
file_path = 'Behavioral data/Survey_Visual_2019-07-23-15-41-47.csv'
data = pd.read_csv(file_path, skiprows=1)
data.columns = data.columns.str.strip().str.lower()

# Normalize values
data['event'] = data['event'].str.strip().str.lower()
data['trial'] = data['trial'].astype(int)

# Containers
success_path_lengths = []
fail_path_lengths = []

trial_numbers = data['trial'].unique()

for trial_id in trial_numbers:
    trial_data = data[data['trial'] == trial_id]
    events = trial_data['event'].values

    # Find start event
    if 'control applied' in events:
        start_idx = trial_data[trial_data['event'] == 'control applied'].index.min()
    elif 'iti start' in events:
        start_idx = trial_data[trial_data['event'] == 'iti start'].index.min()
    else:
        continue  # Skip this trial

    # Find end event
    if 'target reached' in events:
        end_idx = trial_data[trial_data['event'] == 'target reached'].index.min()
        trial_type = 'success'
    elif 'timeout start' in events:
        end_idx = trial_data[trial_data['event'] == 'timeout start'].index.min()
        trial_type = 'fail'
    else:
        continue

    # Calculate path length
    if pd.notna(start_idx) and pd.notna(end_idx) and end_idx > start_idx:
        start_pos = trial_data.loc[start_idx, 'position']
        end_pos = trial_data.loc[end_idx, 'position']
        path_length = abs(end_pos - start_pos)

        if trial_type == 'success':
            success_path_lengths.append(path_length)
        else:
            fail_path_lengths.append(path_length)

# Compute averages
avg_success_path = np.mean(success_path_lengths) if success_path_lengths else np.nan
avg_fail_path = np.mean(fail_path_lengths) if fail_path_lengths else np.nan

# Print results
print("Success trial path lengths:", success_path_lengths)
print("Average success path length:", avg_success_path)
print("Fail trial path lengths:", fail_path_lengths)
print("Average fail path length:", avg_fail_path)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load CSV file
df = pd.read_csv("reward_analysis_results_extended.csv")

# Clean and prepare data
df['Ensemble'] = pd.to_numeric(df['Ensemble'], errors='coerce')
df = df.dropna(subset=['Ensemble'])
ensembles = sorted(df['Ensemble'].unique())
colors = plt.cm.tab20(np.linspace(0, 1, len(ensembles)))

# Compute per-ensemble stats
summary = {
    'Ensemble': [],
    'AvgSuccessfulRate': [],
    'SESuccessfulRate': [],
    'AvgRewardFrequency': [],
    'SERewardFrequency': [],
    'AvgTrialTime': [],
    'SETrialTime': [],
    'N': []
}

for ens in ensembles:
    group = df[df['Ensemble'] == ens]
    n = len(group)
    summary['Ensemble'].append(ens)
    summary['AvgSuccessfulRate'].append(group['SuccessfulRate'].mean())
    summary['SESuccessfulRate'].append(group['SuccessfulRate'].std(ddof=1) / np.sqrt(n))
    summary['AvgRewardFrequency'].append(group['RewardFrequencyPerMin'].mean())
    summary['SERewardFrequency'].append(group['RewardFrequencyPerMin'].std(ddof=1) / np.sqrt(n))
    summary['AvgTrialTime'].append(group['AverageTrialTime'].mean())
    summary['SETrialTime'].append(group['AverageTrialTime'].std(ddof=1) / np.sqrt(n))
    summary['N'].append(n)

summary_df = pd.DataFrame(summary)

# Helper for plotting
def plot_with_errorbars(x, y, yerr, ylabel, title, n_labels, ylim=None):
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(x, y, yerr=yerr, capsize=5, color=colors[:len(x)])
    for i, (xi, yi, err, n) in enumerate(zip(x, y, yerr, n_labels)):
        ax.text(xi, yi + err + 0.03 * max(y), f"n = {n}", ha='center', fontsize=10)
    ax.set_xlabel("Ensemble")
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    if ylim:
        ax.set_ylim(ylim)
    ax.set_xticks(x)
    fig.tight_layout()
    plt.show()

# Plot 1: Success Rate
plot_with_errorbars(
    summary_df['Ensemble'],
    summary_df['AvgSuccessfulRate'],
    summary_df['SESuccessfulRate'],
    "Success Rate",
    "Success Rate by Ensemble",
    summary_df['N'],
    ylim=(0, 1.1)
)

# Plot 2: Reward Frequency
plot_with_errorbars(
    summary_df['Ensemble'],
    summary_df['AvgRewardFrequency'],
    summary_df['SERewardFrequency'],
    "Reward Frequency (per min)",
    "Reward Frequency by Ensemble",
    summary_df['N'],
    ylim=(0, max(summary_df['AvgRewardFrequency']) + 2)
)

# Plot 3: Average Trial Time
plot_with_errorbars(
    summary_df['Ensemble'],
    summary_df['AvgTrialTime'],
    summary_df['SETrialTime'],
    "Average Trial Time (s)",
    "Average Trial Time by Ensemble",
    summary_df['N'],
    ylim=(0, max(summary_df['AvgTrialTime']) + 5)
)
